<a href="https://colab.research.google.com/github/Matej1011/IOU-eval/blob/main/CT_preprocessing_1024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install connected-components-3d SimpleITK nibabel scikit-image tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.4/52.4 MB 13.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!rm -r sample_data/

Mounted at /content/drive


Below is code copy-pasted from https://github.com/bowang-lab/MedSAM/blob/a7b77769ff12035414d0aaf3bc87230b7c10f922/pre_CT_MR.py

In [ ]:
# -*- coding: utf-8 -*-
# %% import packages
# pip install connected-components-3d
import numpy as np

# import nibabel as nib
import SimpleITK as sitk
import os

join = os.path.join
from skimage import transform
from tqdm import tqdm
import cc3d

# convert nii image to npz files, including original image and corresponding masks
modality = "CT"
anatomy = "Abd"  # anatomy + dataset name
img_name_suffix = ".nii.gz"
gt_name_suffix = ".nii"
prefix = modality + "_" + anatomy + "_"


#THIS IS WHERE I EDITED THE PATHS:


nii_path = "/content/drive/MyDrive/SR/rsna-cspine-segmentations/nifti_source_files_segmentations"  # path to the nii images
gt_path = "/content/drive/MyDrive/SR/rsna-cspine-segmentations/segmentations"  # path to the ground truth
npy_path = "data/npy/" + prefix[:-1]
os.makedirs(join(npy_path, "gts"), exist_ok=True)
os.makedirs(join(npy_path, "imgs"), exist_ok=True)

#/content/drive/MyDrive/SR/rsna-cspine-segmentations/nifti_source_files_segmentations/1.2.826.0.1.3680043.10633
#/content/drive/MyDrive/SR/rsna-cspine-segmentations/segmentations/1.2.826.0.1.3680043.10633.nii

image_size = 1024
voxel_num_thre2d = 100
voxel_num_thre3d = 1000

names = sorted(os.listdir(gt_path))
print(f"ori # files {len(names)=}")
names = [
    name
    for name in names
    if os.path.exists(join(nii_path, name.split(gt_name_suffix)[0] + "/" + name.split(gt_name_suffix)[0] + img_name_suffix))
]
print(f"after sanity check # files {len(names)=}")



# set label ids that are excluded
remove_label_ids = [
    12
]  # remove deodenum since it is scattered in the image, which is hard to specify with the bounding box
tumor_id = None  # only set this when there are multiple tumors; convert semantic masks to instance masks
# set window level and width
# https://radiopaedia.org/articles/windowing-ct
WINDOW_LEVEL = 40  # only for CT images
WINDOW_WIDTH = 400  # only for CT images

random_important_integer = 0

# %% save preprocessed images and masks as npz files
for name in tqdm(names[:87]):  # use the remaining 10 cases for validation
  #print("Name: ", name.split(gt_name_suffix)[0] + "/" + name.split(gt_name_suffix)[0] + img_name_suffix, " index: ", random_important_integer)
  if random_important_integer == 8:

    #edited
    image_name = name.split(gt_name_suffix)[0] + "/" + name.split(gt_name_suffix)[0] + img_name_suffix

    gt_name = name
    gt_sitk = sitk.ReadImage(join(gt_path, gt_name))
    gt_data_ori = np.uint8(sitk.GetArrayFromImage(gt_sitk))

    #print("rii, gt_name: ", random_important_integer, gt_name)
    # remove label ids
    for remove_label_id in remove_label_ids:
        gt_data_ori[gt_data_ori == remove_label_id] = 0
    # label tumor masks as instances and remove from gt_data_ori
    if tumor_id is not None:
        tumor_bw = np.uint8(gt_data_ori == tumor_id)
        gt_data_ori[tumor_bw > 0] = 0
        # label tumor masks as instances
        tumor_inst, tumor_n = cc3d.connected_components(
            tumor_bw, connectivity=26, return_N=True
        )
        # put the tumor instances back to gt_data_ori
        gt_data_ori[tumor_inst > 0] = (
            tumor_inst[tumor_inst > 0] + np.max(gt_data_ori) + 1
        )

    # exclude the objects with less than 1000 pixels in 3D
    gt_data_ori = cc3d.dust(
        gt_data_ori, threshold=voxel_num_thre3d, connectivity=26, in_place=True
    )
    # remove small objects with less than 100 pixels in 2D slices

    for slice_i in range(gt_data_ori.shape[0]):
        gt_i = gt_data_ori[slice_i, :, :]
        # remove small objects with less than 100 pixels
        # reason: fro such small objects, the main challenge is detection rather than segmentation
        gt_data_ori[slice_i, :, :] = cc3d.dust(
            gt_i, threshold=voxel_num_thre2d, connectivity=8, in_place=True
        )
    # find non-zero slices
    z_index, _, _ = np.where(gt_data_ori > 0)
    z_index = np.unique(z_index)

    if len(z_index) > 0:
        # crop the ground truth with non-zero slices
        gt_roi = gt_data_ori[z_index, :, :]
        # load image and preprocess
        img_sitk = sitk.ReadImage(join(nii_path, image_name))
        image_data = sitk.GetArrayFromImage(img_sitk)
        # nii preprocess start
        if modality == "CT":
            lower_bound = WINDOW_LEVEL - WINDOW_WIDTH / 2
            upper_bound = WINDOW_LEVEL + WINDOW_WIDTH / 2
            image_data_pre = np.clip(image_data, lower_bound, upper_bound)
            image_data_pre = (
                (image_data_pre - np.min(image_data_pre))
                / (np.max(image_data_pre) - np.min(image_data_pre))
                * 255.0
            )
        else:
            lower_bound, upper_bound = np.percentile(
                image_data[image_data > 0], 0.5
            ), np.percentile(image_data[image_data > 0], 99.5)
            image_data_pre = np.clip(image_data, lower_bound, upper_bound)
            image_data_pre = (
                (image_data_pre - np.min(image_data_pre))
                / (np.max(image_data_pre) - np.min(image_data_pre))
                * 255.0
            )
            image_data_pre[image_data == 0] = 0

        image_data_pre = np.uint8(image_data_pre)
        img_roi = image_data_pre[z_index, :, :]

        img_roi = np.transpose(img_roi, (2, 1, 0))
        img_1024 = transform.resize(img_roi, (1024, 1024), order=3, preserve_range=True, anti_aliasing=True).astype(np.uint8)
        img_1024 = np.transpose(img_1024, (2, 1, 0))

        np.savez_compressed(join(npy_path, prefix + gt_name.split(gt_name_suffix)[0]+'.npz'), imgs=img_1024, gts=gt_roi, spacing=img_sitk.GetSpacing())

        # save the image and ground truth as nii files for sanity check;
        # they can be removed
        #img_roi_sitk = sitk.GetImageFromArray(img_roi)
        #gt_roi_sitk = sitk.GetImageFromArray(gt_roi)

  random_important_integer += 1
  #commented out:


ori # files len(names)=87
after sanity check # files len(names)=87


  0%|          | 0/87 [00:00<?, ?it/s]

rii, gt_name:  8 1.2.826.0.1.3680043.14267.nii


100%|██████████| 87/87 [04:11<00:00,  2.89s/it]


In [ ]:
import shutil
from google.colab import files
shutil.make_archive('preprocessed_62-87_npz_patients', 'zip', '/content/data/npy/CT_Abd')

'/content/preprocessed_62-87_npz_patients.zip'

In [ ]:
files.download('preprocessed_62-87_npz_patients.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>